# Bernoulli Console with Random Geometry

In [1]:
seed = 0  # integer or None
seed = None

In [2]:
import numpy as np

rs = np.random.RandomState()
if isinstance(seed, int):
    rs.seed(seed)


def brownian1d(T=1, N=100, mu=0.1, sigma=0.01, S0=20):
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = rs.standard_normal(size=N)
    W = np.cumsum(W)*np.sqrt(dt)  # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W
    S = S0*np.exp(X)  # geometric brownian motion
    return S


params = dict(T=0.1, N=100, mu=0.5, sigma=0.1)

x = brownian1d(**params)
y = brownian1d(**params)
z = brownian1d(**params)


In [3]:
from polymesh import PolyData, CartesianFrame, LineData
import numpy as np

frame = CartesianFrame(dim=3)

coords = np.stack([x, y, z], axis=1)
coords[:] -= coords[0]
coords = np.vstack((coords, np.array([1., 0., 0.])))

topo = np.zeros((coords.shape[0]-1, 2)).astype(int)
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1

mesh = PolyData(coords=coords, frame=frame)
mesh['lines'] = LineData(topo=topo, frame=frame)

mesh.plot(notebook=False)


Index and coordinate of the last node. Later, when the 2-noded mesh is transformed into a mesh of 3-noded elements, this information is going to be necessary to identify the free end.

In [4]:
i_last = len(coords) - 1
i_last, coords[i_last]

(100, array([1., 0., 0.]))

The parameters of the following cell are up for manipulation. They define a simple rectangular cross section and an isotropic Hooke model governed by Young's modulus and Poisson's ratio.

In [5]:
# INPUT DATA
w, h = 10., 10.  # width and height of the rectangular cross section
F = -100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

The model stiffness matrix is calculated automatically here:

In [6]:
import numpy as np
from numpy import pi as PI

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

## Solution with 2-Node Bernoulli Element

Run a linear analysis and do some postprocessing using 2-Noded Bernoulli members.

In [7]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from sigmaepsilon.solid.fem.cells import B2 as Beam
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.space.utils import index_of_closest_point

# space
GlobalFrame = StandardFrame(dim=3)

i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
i_last = index_of_closest_point(coords, np.array([1., 0., 0.]))

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[i_last, :3] = global_load_vector
fixity[i_first, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution(store='dofsol')
forces = structure.nodal_forces()
reactions = structure.reaction_forces()
internal_forces = structure.internal_forces()

In [8]:
coords[i_first], coords[i_last]

(array([0., 0., 0.]), array([1., 0., 0.]))

In [9]:
frame0 = StandardFrame(frames[0])
RZ = reactions[i_first, 2]
RYY = reactions[i_first, 4]
FZ = Vector(internal_forces[0, 0, :3], frame=frame0).show(GlobalFrame)[2]
FYY = Vector(internal_forces[0, 0, 3:], frame=frame0).show(GlobalFrame)[1]
(F, FZ, RZ), (FYY, RYY)

((-100.0, -99.99999973788547, 99.99999973788726),
 (99.99999972967505, -99.99999972967458))

## Solution with 3-Node Bernoulli Elements

We transform the mesh using the appropriate transformation function from `PolyMesh`.

In [10]:
from polymesh.topo.tr import L2_to_L3
coords, topo = L2_to_L3(coords, topo)

Rebuild and calculate the mesh using 3-Node elements.

In [11]:
from sigmaepsilon.solid.fem.cells import B3 as Beam

i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
i_last = index_of_closest_point(coords, np.array([1., 0., 0.]))

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[i_last, :3] = global_load_vector
fixity[i_first, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution(store='dofsol')
forces = structure.nodal_forces()
reactions = structure.reaction_forces()
internal_forces = structure.internal_forces()

In [12]:
coords[i_first], coords[i_last]

(array([0., 0., 0.]), array([1., 0., 0.]))

In [13]:
frame0 = StandardFrame(frames[0])
RZ = reactions[i_first, 2]
RYY = reactions[i_first, 4]
FZ = Vector(internal_forces[0, 0, :3], frame=frame0).show(GlobalFrame)[2]
FYY = Vector(internal_forces[0, 0, 3:], frame=frame0).show(GlobalFrame)[1]
(F, FZ, RZ), (FYY, RYY)

((-100.0, -99.99998838475845, 99.99998838406736),
 (99.99999945374424, -99.99999945374162))